In [4]:
# -*- coding:utf-8 -*-

from __future__ import division
import tkinter as tk
from tkinter import *
import tkinter.messagebox as messagebox
from PIL import Image, ImageTk
import os
import cv2
import numpy as np


class LabelTool():
    def __init__(self, master):
        # 初始化变量
        # 图片1初始路径
        self.image1_dir = ''
        # 图片2初始路径
        self.image2_dir = ''
        # 标签记录1
        self.label1_record = []
        # 标签记录2
        self.label2_record = []
        # 图片1状态
        self.image1 = None
        # 图片2状态
        self.image2 = None
        # 状态1, 0代表未转换，1代表已转换
        self.sta1 = 0
        # 状态2
        self.sta2 = 0

        # 设置主要框架
        self.parent = master
        self.parent.title("TPS融合")
        self.frame = tk.Frame(self.parent)
        self.frame.pack(fill=tk.BOTH, expand=1)
        self.parent.resizable(width=TRUE, height=TRUE)

        # 图片加载
        self.entry = tk.Entry(self.frame)
        self.entry.grid(row=0, column=0, sticky=E)
        self.ldBtn = tk.Button(self.frame, text="Loading", command=self.loadImage1)
        self.ldBtn.grid(row=0, column=1, sticky=W)

        self.entry2 = tk.Entry(self.frame)
        self.entry2.grid(row=0, column=2, sticky=E)
        self.ldBtn2 = tk.Button(self.frame, text="Loading", command=self.loadImage2)
        self.ldBtn2.grid(row=0, column=3, sticky=W)

        # 显示窗口
        self.mainPanel = Canvas(self.frame, cursor='arrow')
        self.mainPanel.bind("<Button-1>", self.mouseClick1)
        self.mainPanel.grid(row=1, column=0, rowspan=4, sticky=NW)

        self.mainPanel2 = Canvas(self.frame, cursor='arrow')
        self.mainPanel2.bind("<Button-1>", self.mouseClick2)
        self.mainPanel2.grid(row=1, column=2, rowspan=4, sticky=NW)

        # 控制和显示台
        self.ctrPanel = Frame(self.frame)
        self.ctrPanel.grid(row=5, column=0, columnspan=4, sticky=W + E)
        self.clearBtn = Button(self.ctrPanel, text='Clear', width=10, command=self.clear1)
        self.clearBtn.grid(row=5, column=0, sticky=W)
        self.conBtn = Button(self.ctrPanel, text='TPS Convert', width=10, command=self.conver1)
        self.conBtn.grid(row=5, column=1, sticky=W)

        self.ctrPanel2 = Frame(self.frame)
        self.ctrPanel2.grid(row=5, column=2, columnspan=4, sticky=W + E)
        self.clearBtn2 = Button(self.ctrPanel2, text='Clear', width=10, command=self.clear2)
        self.clearBtn2.grid(row=5, column=0, sticky=W)
        self.conBtn2 = Button(self.ctrPanel2, text='TPS Convert', width=10, command=self.conver2)
        self.conBtn2.grid(row=5, column=1, sticky=W)

    def loadImage1(self):
        """
        加载图片1
        :return:
        """
        or_dir = self.entry.get()
        or_dir = or_dir.replace('\\', '/')
        # 路径是否存在
        if not os.path.exists(or_dir):
            messagebox.showerror("Error!", message = "路径不存在")
        else:
            pil_image = Image.open(or_dir)
            self.image1 = ImageTk.PhotoImage(pil_image)
            self.mainPanel.config(width=max(self.image1.width(), 512), height=max(self.image1.height(), 512))
            self.mainPanel.create_image(0, 0, image=self.image1, anchor=NW)

    def loadImage2(self):
        """
        加载图片2
        :return:
        """
        or_dir = self.entry2.get()
        or_dir = or_dir.replace('\\', '/')
        # 路径是否存在
        if not os.path.exists(or_dir):
            messagebox.showerror("Error!", message = "路径不存在")
        else:
            pil_image = Image.open(or_dir)
            # 加载图像
            self.image2 = ImageTk.PhotoImage(pil_image)
            self.mainPanel2.config(width=max(self.image2.width(), 512), height=max(self.image2.height(), 512))
            self.mainPanel2.create_image(0, 0, image=self.image2, anchor=NW)

    def mouseClick1(self, event):
        """
        鼠标点击动作，增加一个记录
        :param event:
        :return:
        """
        self.label1_record.append([event.x, event.y])
        self.mainPanel.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, fill="red", tag="r1")

    def mouseClick2(self, event):
        """
        鼠标点击动作，增加一个记录
        :param event:
        :return:
        """
        self.label2_record.append([event.x, event.y])
        self.mainPanel2.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, fill="red", tag="r2")

    def clear1(self):
        """
        清除图片1标注点和记录
        :return:
        """
        self.label1_record=[]
        self.mainPanel.delete("r1")

    def clear2(self):
        """
        清除图片2的记录
        :return:
        """
        self.label2_record=[]
        self.mainPanel2.delete("r2")

    def conver1(self):
        """
        图片TPS变换
        :return:
        """
        n1 = len(self.label1_record)
        n2 = len(self.label2_record)
        if n1 == n2:
            tps = cv2.createThinPlateSplineShapeTransformer()
            sshape = np.array(self.label1_record, np.int32)
            tshape = np.array(self.label2_record, np.int32)
            sshape = sshape.reshape(1, -1, 2)
            tshape = tshape.reshape(1, -1, 2)

            matches = list()
            for i in range(n1):
                matches.append(cv2.DMatch(i, i, 0))
            tps.estimateTransformation(tshape, sshape, matches)
            or_dir = self.entry.get()
            or_dir = or_dir.replace('\\', '/')
            img2_o = cv2.imread(or_dir)
            out_img = tps.warpImage(img2_o)
            cv2.imshow('1', out_img)


    def conver2(self):
        """
        图片TPS变换
        :return:
        """
        n1 = len(self.label1_record)
        n2 = len(self.label2_record)
        if n1 == n2:
            tps = cv2.createThinPlateSplineShapeTransformer()
            sshape = np.array(self.label2_record, np.int32)
            tshape = np.array(self.label1_record, np.int32)
            sshape = sshape.reshape(1, -1, 2)
            tshape = tshape.reshape(1, -1, 2)

            matches = list()
            for i in range(n2):
                matches.append(cv2.DMatch(i, i, 0))
            tps.estimateTransformation(tshape, sshape, matches)
            or_dir = self.entry2.get()
            or_dir = or_dir.replace('\\', '/')
            img2_o = cv2.imread(or_dir)
            out_img = tps.warpImage(img2_o)
            cv2.imshow('1', out_img)


root = tk.Tk()
tool = LabelTool(root)
root.mainloop()